近日工作流程

1.在不同的feature map大小下测试ESPCN模型

2.改造VESPCN模型

3.熟悉Zero shot模型

4.改进data loader

把数据加载器加进来

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /
%cd content/drive/MyDrive/Colab Notebooks/CT_project/

Mounted at /content/drive
/
/content/drive/MyDrive/Colab Notebooks/CT_project


In [2]:
#在这里需要开始运行处理程序
#这一步处理程序中，可能还是需要分割，但是需要注意的是，分割的图片需要同时运用于两个子模型上
import cv2 
import numpy as np
import os,sys
import shutil
from __future__ import absolute_import
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.autograd import Variable
import torchvision
from torch.nn import init
import functools
import shutil
from torch.utils.data import Dataset,DataLoader,TensorDataset
all_depth=256
all_cnt=0
train_batch=64
split_scale_H=4
split_scale_W=8
find_width=None
find_height=None
frame_num=3
scale=4
#顺序2：一个文件夹下的图片是一个物体的多个截面，读取一个文件夹下的图片
#最开始的图片的排列顺序是width*heigth
def read_path(file_pathname):
    #遍历该目录下的所有图片文件
    img_list=[]
    files=os.listdir(file_pathname)
    tem_files=[]
    for o in files:
        if(not o[0]=='.'):
            tem_files.append(o)
    files=tem_files
    files.sort(key= lambda x:int(x[-9:-4]))
    cnt=0
    for filename in files:
        if(cnt==all_depth):
          break
        img = cv2.imread(file_pathname+'/'+filename)
        img_list.append(img[:,:,0])
        cnt=cnt+1
    print(("read the model"+file_pathname+",there are files:",len(img_list)))
    return img_list
#顺序3：将read_path得到的若干张图片进行纵向切割
#图片数量只去all_depth张
#局部变量：width图片的宽度，height图片的长度，depth是图片的数量（高度）
#         new_img_list1的切割方向是生成depth*width，而new_img_list2的切割方向是生成depth*height
#全局变量：find_height,find_width传递width和height，scale是对其降采样的倍数。
#注意：再这里还没有产生video
def downsampling_mean(img_list,down_scale):
    new_img_list=[]
    for i in range(0,len(img_list)):
        img,k=img_list[i].copy(),0
        for j in range(0,img_list[i].shape[0],down_scale):
            new_line=np.sum(img_list[i][j:j+down_scale,:],axis=0)/down_scale
            img[k,:]=new_line.copy()
            k=k+1
        new_img_list.append(img[0:k,:].copy())
    return new_img_list
def generate_image(dir_path):
    img_list=read_path(dir_path)
    if(len(img_list)<all_depth):
        print("invalid image list for too few images")
        return [],[]
    width,height=img_list[0].shape[0],img_list[0].shape[1]
    global find_height,find_width;
    find_height,find_width=height,width
    depth=all_depth #len(img_list)#这个地方貌似有点问题?
    new_img_list1,new_img_list2=[],[]
    for i in range(0,height):
        img = np.zeros((depth,width), np.uint8)
        new_img_list1.append(img)
    for i in range(0,width):
        img = np.zeros((depth,height), np.uint8)
        new_img_list2.append(img)
    for j in range(0,depth):
        for i in range(0,height):
            img=img_list[j]
            new_img_list1[i][j,0:width]=img[0:width,i].reshape(1,width).copy()
    for j in range(0,depth):
        for i in range(0,width):
            img=img_list[j]
            new_img_list2[i][j,0:height]=img[i,0:height].copy()
    #这里做down-sampling，应该需要通过函数来做
    new_img_list3,new_img_list4=downsampling_mean(new_img_list1,scale),downsampling_mean(new_img_list2,scale)
    return (new_img_list1,new_img_list2,new_img_list3,new_img_list4)
#  """
#  在这一个函数中,需要将new_img_list进行裁剪,裁剪比例为split_scale_H与split_scale_W
#  可以发现裁剪过后的结果它的顺序是行排除的.，先把一行排满了，然后再来排一列（实际上是先再升读上取）
#所以恢复的时候，需要先将它们给叠起来，然后再排列行，然后再排列列。
#  """
#这里所说的H实际上对应的是depth，而W对应的width和heigth那一维
#局部变量：true_height_sub，true_width_sub是裁剪过后的子图的大小，true_height,true_width是原图的大小

def split_image(new_img_list,zoomin=False):
    print(new_img_list[0].shape)
    split_img_list=[]
    height,width=new_img_list[0].shape
    true_height_sub,true_width_sub=int(height/split_scale_H),int(width/split_scale_W)
    true_height,true_width=height,width
    for j in range(0,true_height,true_height_sub):
        for k in range(0,true_width,true_width_sub):
            for i in range(0,len(new_img_list)-2):
                sub_list=[]
                for ii in range(i,i+frame_num):
                    #在这里需要对img_list进行分割
                    img=new_img_list[ii][j:j+true_height_sub,k:k+true_width_sub].copy()
                    sub_list.append(img)
                split_img_list.append(sub_list)
    return split_img_list
def toString(i):
    if(i<10):
        return '000'+str(i)
    elif(i<100):
        return '00'+str(i)
    elif(i<1000):
        return '0'+str(i)
    else:
        return str(i)
def remove_dir(dir):
    dir = dir.replace('\\', '/')
    if(os.path.isdir(dir)):
        for p in os.listdir(dir):
            remove_dir(os.path.join(dir,p))
        if(os.path.exists(dir)):
            os.rmdir(dir)
    else:
        if(os.path.exists(dir)):
            os.remove(dir)
def write_dataset(file_dir_path):
    for dir in os.listdir(file_dir_path):
        all_dir=file_dir_path+'/'+dir
        print(all_dir)
        if(os.path.isdir(all_dir)):
            continue
        new_img_list1,new_img_list2,new_img_list3,new_img_list4=generate_image(all_dir)
        split_img_list1=split_image(new_img_list1)
        split_img_list2=split_image(new_img_list2)
        split_img_list3=split_image(new_img_list3,True)
        split_img_list4=split_image(new_img_list4,True)
        if(os.path.exists("./dataset_new/"+dir)):
            shutil.rmtree("./dataset_new/"+dir)
        os.makedirs("./dataset_new/"+dir)
        for i in range(len(split_img_list1)):
            sub_dir="./dataset_new/"+dir+"/a"+toString(i)
            os.makedirs(sub_dir)
            for j in range(len(split_img_list1[i])):
                filename=sub_dir+'/'+toString(j)+'.jpg'
                cv2.imwrite(filename,split_img_list1[i][j])
        for i in range(len(split_img_list2)):
            sub_dir="./dataset_new/"+dir+"/b"+toString(i)
            os.makedirs(sub_dir)
            for j in range(len(split_img_list2[i])):
                filename=sub_dir+'/'+toString(j)+'.jpg'
                cv2.imwrite(filename,split_img_list2[i][j])
        for i in range(len(split_img_list3)):
            sub_dir="./dataset_new/"+dir+"/c"+toString(i)
            os.makedirs(sub_dir)
            for j in range(len(split_img_list3[i])):
                filename=sub_dir+'/'+toString(j)+'.jpg'
                cv2.imwrite(filename,split_img_list3[i][j])
        for i in range(len(split_img_list4)):
            sub_dir="./dataset_new/"+dir+"/d"+toString(i)
            os.makedirs(sub_dir)
            for j in range(len(split_img_list4[i])):
                filename=sub_dir+'/'+toString(j)+'.jpg'
                cv2.imwrite(filename,split_img_list4[i][j])
#---------------------------------------------------------------------------------------------#
#---------------------------------------------------------------------------------------------#
"""
1.在这里需要对其进行裁剪,记裁剪比例为
2.除了test之外,还应该记录一个新的dataset,用来显示效果,并且它需要排布整齐,大小为模型的倍数
display_dataset

"""
#---------------------------------------------------------------------------------------------#
#---------------------------------------------------------------------------------------------#
#这里的all_cnt记录了总共需要读多少组数据，因为数据量太大加载起来费时，所以测试阶段all_cnt==4
#再最开始生成src和target的步骤中，先不考虑display，display是再shuffle的过程中才开始考虑的。
def dataload(file_dir_path,is_image=False):
    dataset_src=[]      #list of tensor[T,1,H,W]
    dataset_target=[]    #list of tensor[T,1,H,W]
    for dir in os.listdir(file_dir_path):
        #在这里首先需要判断是否是文件夹
        all_dir=file_dir_path+'/'+dir
        if(dir[0]=='.' or (not os.path.isdir(all_dir))):
            continue
        global all_cnt;
        if(all_cnt>=4):#它记录了总共需要读多少个文件
            break
        all_cnt=all_cnt+1
        print(all_dir)
        new_img_list1,new_img_list2,new_img_list3,new_img_list4=generate_image(all_dir)
        split_img_list1,split_img_list2,split_img_list3,split_img_list4=split_image(new_img_list1),split_image(new_img_list2),split_image(new_img_list3,True),split_image(new_img_list4,True)
        for i in range(len(split_img_list1)):
            oneVideo=[]
            for j in range(len(split_img_list1[i])):
                tem=torch.from_numpy(split_img_list1[i][j])
                tem=tem.unsqueeze(0)
                oneVideo.append(tem)
            dataset_src.append(torch.stack(oneVideo))
        for i in range(len(split_img_list3)):
            oneVideo=[]
            for j in range(len(split_img_list3[i])):
                tem=torch.from_numpy(split_img_list3[i][j])
                tem=tem.unsqueeze(0)
                oneVideo.append(tem)
            dataset_target.append(torch.stack(oneVideo))
    #print(dataset_src)
    #在这里需要对dataset_target和dataset_src进行类型转化
    #这里需要计算display_dataset的大小
    #同时需要注意的是train_dataset中,它是先按一个方向切割,这个方向排布完成后再排布另一个方向
    #display_for_one是为了计算一个模型需要多少个video对其进行对应，model_num是一个需要去多少个物体
    display_for_one,model_num=split_scale_H*split_scale_W*(find_width-2),1
    display_size=display_for_one*model_num
    test_dataset_size=display_size
    print(len(dataset_target))
    train_dataset_src,train_dataset_target=torch.stack(dataset_target[0:-test_dataset_size]),torch.stack(dataset_src[0:-test_dataset_size])
    train_dataset_src,train_dataset_target=train_dataset_src.float()/255.0,train_dataset_target.float()/255.0
    display_dataset_src,display_dataset_target=torch.stack(dataset_target[-display_size:]),torch.stack(dataset_src[-display_size:])
    display_dataset_src,display_dataset_target=display_dataset_src.float()/255.0,display_dataset_target.float()/255.0
    print(train_dataset_src.shape)
    print(train_dataset_target.shape);
    if is_image:
        train_dataset_src=torch.flatten(train_dataset_src, start_dim=0, end_dim=1)
        train_dataset_target=torch.flatten(train_dataset_target, start_dim=0, end_dim=1)
        display_dataset_src=torch.flatten(display_dataset_src, start_dim=0, end_dim=1)
        display_dataset_target=torch.flatten(display_dataset_target, start_dim=0, end_dim=1)

    train_dataset=TensorDataset(train_dataset_src,train_dataset_target)
    display_dataset=TensorDataset(display_dataset_src,display_dataset_target)
    test_dataset_src,test_dataset_target=torch.stack(dataset_target[-test_dataset_size:]),torch.stack(dataset_src[-test_dataset_size:])
    test_dataset_src,test_dataset_target=test_dataset_src.float()/255.0,test_dataset_target.float()/255.0
    print("train_dataset:")
    print((train_dataset_src.shape,train_dataset_target.shape))
    print("display_dataset:")
    print((display_dataset_src.shape,display_dataset_target.shape))
    print("test_dataset:")
    print((test_dataset_src.shape,test_dataset_target.shape))
    if is_image:
        test_dataset_src=torch.flatten(test_dataset_src, start_dim=0, end_dim=1)
        test_dataset_target=torch.flatten(test_dataset_target, start_dim=0, end_dim=1)
    test_dataset=TensorDataset(test_dataset_src,test_dataset_target)
    
    return train_dataset,test_dataset,display_dataset

In [3]:
all_cnt=0
file_dir_path="./data"		
train_dataset,test_dataset,display_dataset=dataload(file_dir_path)
train_loader = DataLoader(dataset=train_dataset,
                batch_size=train_batch,
                shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                batch_size=train_batch,
                shuffle=True)
display_loader=DataLoader(dataset=display_dataset,
                batch_size=train_batch,
                shuffle=False)
#display也是被切割成batch了的,然后后面需要对其进行拼接

./data/C099
('read the model./data/C099,there are files:', 256)
(256, 512)
(256, 512)
(64, 512)
(64, 512)
./data/C081
('read the model./data/C081,there are files:', 256)
(256, 512)
(256, 512)
(64, 512)
(64, 512)
./data/C050
('read the model./data/C050,there are files:', 256)
(256, 512)
(256, 512)
(64, 512)
(64, 512)
./data/C067
('read the model./data/C067,there are files:', 256)
(256, 512)
(256, 512)
(64, 512)
(64, 512)
65280
torch.Size([48960, 3, 1, 16, 64])
torch.Size([48960, 3, 1, 64, 64])
train_dataset:
(torch.Size([48960, 3, 1, 16, 64]), torch.Size([48960, 3, 1, 64, 64]))
display_dataset:
(torch.Size([16320, 3, 1, 16, 64]), torch.Size([16320, 3, 1, 64, 64]))
test_dataset:
(torch.Size([16320, 3, 1, 16, 64]), torch.Size([16320, 3, 1, 64, 64]))


In [9]:
list=[nn.Conv2d(1, 64, kernel_size=3, padding=1, bias=True),]
list2=[list[0],]
print(id(list[0]))
print(id(list2[0]))
"""a=torch.tensor([[[1,2,3],[3,4,5]],[[1,2,3],[3,4,5]]],dtype=float)
print(a)
print(a.shape)
a[:,0]/=2;
print(a)
print(a.shape)
"""

139852404944592
139852404944592


'a=torch.tensor([[[1,2,3],[3,4,5]],[[1,2,3],[3,4,5]]],dtype=float)\nprint(a)\nprint(a.shape)\na[:,0]/=2;\nprint(a)\nprint(a.shape)\n'

1.改造VESPCN模型

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# TODO: Is the first channel flow with respect to x?
# TODO: Fix 'mean' issues


class Approx_Huber_Loss(nn.Module):
    def __init__(self, args):
        super(Approx_Huber_Loss, self).__init__()
        self.device = torch.device('cpu' if args.cpu else 'cuda')
        self.sobel_filter_X = np.array([[1, 0, -1], [2, 0, -2], [1, 0, -1]]).reshape((1, 1, 3, 3))
        self.sobel_filter_Y = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]]).reshape((1, 1, 3, 3))
        self.sobel_filter_X = torch.from_numpy(self.sobel_filter_X).float().to(self.device)
        self.sobel_filter_Y = torch.from_numpy(self.sobel_filter_Y).float().to(self.device)
        self.epsilon = torch.Tensor([0.01]).float().to(self.device)

    def forward(self, flow):
        flow_X = flow[:, 0:1]
        flow_Y = flow[:, 1:]
        grad_X = F.conv2d(flow_X, self.sobel_filter_X, bias=None, stride=1, padding=1)
        grad_Y = F.conv2d(flow_Y, self.sobel_filter_Y, bias=None, stride=1, padding=1)
        huber = torch.sqrt(self.epsilon + torch.sum(grad_X.pow(2)+grad_Y.pow(2)))
        return huber

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

def make_mc():
    return MotionCompensator()

class MotionCompensator(nn.Module):
    def __init__(self):
        self.device = 'cuda'
        super(MotionCompensator, self).__init__()
        print("Creating Motion compensator")
        #这里的groups就是有多少个并在一起的
        def _gconv(in_channels, out_channels, kernel_size=3, groups=1, stride=1, bias=True):
            return nn.Conv2d(in_channels*groups, out_channels*groups, kernel_size, groups=groups, stride=stride,
                             padding=(kernel_size // 2), bias=bias)

        # Coarse flow
        coarse_flow = [_gconv(2, 24, kernel_size=5, groups=1, stride=2), nn.ReLU(inplace=True)]
        coarse_flow.extend([_gconv(24, 24, kernel_size=3, groups=1), nn.ReLU(True)])
        coarse_flow.extend([_gconv(24, 24, kernel_size=5, groups=1, stride=2), nn.ReLU(True)])
        coarse_flow.extend([_gconv(24, 24, kernel_size=3, groups=1), nn.ReLU(True)])
        coarse_flow.extend([_gconv(24, 32, kernel_size=3, groups=1), nn.Tanh()])
        coarse_flow.extend([nn.PixelShuffle(4)])

        self.C_flow = nn.Sequential(*coarse_flow)

        # Fine flow
        fine_flow = [_gconv(5, 24, kernel_size=5, groups=1, stride=2), nn.ReLU(inplace=True)]
        for _ in range(3):
            fine_flow.extend([_gconv(24, 24, kernel_size=3, groups=1), nn.ReLU(True)])
        fine_flow.extend([_gconv(24, 8, kernel_size=3, groups=1), nn.Tanh()])
        fine_flow.extend([nn.PixelShuffle(2)])

        self.F_flow = nn.Sequential(*fine_flow)

    def forward(self, frame_1, frame_2):
        #frame 的大小为 [N, n_colors, H, W]

        # Create identity flow
        #print(frame_1.shape)
        x = np.linspace(-1, 1, frame_1.shape[3]) # x=W
        y = np.linspace(-1, 1, frame_1.shape[2]) # y=H
        xv, yv = np.meshgrid(x, y)
        #(-1,0,1)
        #(-1,0,1)
        # 3,3,2 也就是说每个格子中存放的是坐标
        #xv: [[-1 0 1]
        #     [-1 0 1]
        #     [-1 0 1]]
        #yv: [[1   1  1]
        #     [0   0  0]
        #     [-1 -1 -1]]
        #
        #
        #
        #stack:https://blog.csdn.net/wgx571859177/article/details/80987459
        id_flow = np.expand_dims(np.stack([xv, yv], axis=-1), axis=0) #（1，W，H，2）
        self.identity_flow = torch.from_numpy(id_flow).float().to(self.device)

        # Coarse flow，它输出的Coarse flow已经变成了原来的四倍了
        coarse_in = torch.cat((frame_1, frame_2), dim=1)
        coarse_out = self.C_flow(coarse_in)
        coarse_out[:,0] /= frame_1.shape[3] #也就说这里，这里第二维及以后的都要除frame_1.shape[3]
        coarse_out[:,1] /= frame_2.shape[2] #但是为什么要这么做？
        frame_2_compensated_coarse = self.warp(frame_2, coarse_out)
        
        # Fine flow
        fine_in = torch.cat((frame_1, frame_2, frame_2_compensated_coarse, coarse_out), dim=1)
        fine_out = self.F_flow(fine_in)
        fine_out[:,0] /= frame_1.shape[3]
        fine_out[:,1] /= frame_2.shape[2]
        flow = (coarse_out + fine_out)

        frame_2_compensated = self.warp(frame_2, flow)

        return frame_2_compensated, flow

    def warp(self, img, flow):
        # https://discuss.pytorch.org/t/solved-how-to-do-the-interpolating-of-optical-flow/5019
        # permute flow N C H W -> N H W C
        # clamp把结果加紧到（-1，1）中
        #
        img_compensated = F.grid_sample(img, (-flow.permute(0,2,3,1)+self.identity_flow).clamp(-1,1), padding_mode='border')
        return img_compensated


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

#args 中包括2个内容：scale与n_sequence
def make_espcn():
    return ESPCN_multiframe2()
#这个类的forward函数待会还要改一下
#fmap的大小是[B,sizeO=scale,H,W]，这里价格H给连起来了
#最后得到的是[B,1,H,W]，这里没有什么问题，因为ESPCN中虽然输出的n_sequence=3，但是输出仍然只有一张
class myPixelShuffle(nn.Module):
    def __init__(self,scale):
        super(myPixelShuffle, self).__init__()
        self.scale=scale
    def forward(self,fmap):
        list=[]
        for i in range(fmap.shape[2]):
            for j in range(fmap.shape[1]):
                list.append(fmap[:,j:j+1,i:i+1,:])
        return torch.cat(list,dim=2)    
def mypixel_shuffle(fmap,scale):
	#这里fmap的维度是:[B,sizeO=scale,H,W],我需要做的是把他们H给连接起来
    list=[]
    for i in range(fmap.shape[2]):
        for j in range(fmap.shape[1]):
            list.append(fmap[:,j:j+1,i:i+1,:])
    return torch.cat(list,dim=2)
#下面的64和差别没有关系，后面可以考虑将其变小，
class ESPCN_multiframe2(nn.Module):
    # Add Residual connection!
    def __init__(self, ):
        super(ESPCN_multiframe2, self).__init__()
        print("Creating ESPCN multiframe2 (x%d)" % scale)
        network = [nn.Conv2d( 3, 64, kernel_size=3, padding=1), nn.ReLU(True)]
        network.extend([nn.Conv2d(64, 64, kernel_size=3, padding=1), nn.ReLU(True)])
        network.extend([nn.Conv2d(64, 64, kernel_size=3, padding=1), nn.ReLU(True)])
        network.extend([nn.Conv2d(64, 32, kernel_size=3, padding=1), nn.ReLU(True)])
        network.extend([nn.Conv2d(32, 32, kernel_size=3, padding=1), nn.ReLU(True)])
        network.extend([nn.Conv2d(32, 32, kernel_size=3, padding=1), nn.ReLU(True)])
        network.extend([nn.Conv2d(32, 20, kernel_size=3, padding=1), nn.ReLU(True)])
        network.extend(
            [nn.Conv2d(20, scale, kernel_size=3, padding=1), nn.ReLU(True)])
        network.extend([myPixelShuffle(scale)])
        network.extend([nn.Conv2d(1, 1, kernel_size=1, padding=0)])
        self.net = nn.Sequential(*network)
    def forward(self, x):
        #if isinstance(x, list):
            #通过阅读下面的代码可以发现，其接受到的数据实际上为[N, n_sequence * n_colors, H, W]=[N,n_colors, H, W]
            #在我们的模型中n_colors=1
            # squeeze frames n_sequence * [N, 1, n_colors, H, W] -> n_sequence * [N, n_colors, H, W]
            #lr_frames_squeezed = [torch.squeeze(frame, dim = 1) for frame in x]
            # concatenate frames n_sequence * [N, n_colors, H, W] -> [N, n_sequence * n_colors, H, W]
            #x = torch.cat(lr_frames_squeezed, dim = 1)    
        return self.net(x)

这里是那里提到了的两个损失函数

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# TODO: Is the first channel flow with respect to x?
# TODO: Fix 'mean' issues


class Approx_Huber_Loss(nn.Module):
    def __init__(self):
        super(Approx_Huber_Loss, self).__init__()
        self.device = torch.device('cuda')
        self.sobel_filter_X = np.array([[1, 0, -1], [2, 0, -2], [1, 0, -1]]).reshape((1, 1, 3, 3))
        self.sobel_filter_Y = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]]).reshape((1, 1, 3, 3))
        self.sobel_filter_X = torch.from_numpy(self.sobel_filter_X).float().to(self.device)
        self.sobel_filter_Y = torch.from_numpy(self.sobel_filter_Y).float().to(self.device)
        self.epsilon = torch.Tensor([0.01]).float().to(self.device)

    def forward(self, flow):
        flow_X = flow[:, 0:1]
        flow_Y = flow[:, 1:]
        grad_X = F.conv2d(flow_X, self.sobel_filter_X, bias=None, stride=1, padding=1)
        grad_Y = F.conv2d(flow_Y, self.sobel_filter_Y, bias=None, stride=1, padding=1)
        huber = torch.sqrt(self.epsilon + torch.sum(grad_X.pow(2)+grad_Y.pow(2)))
        return huber

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
#from model.motioncompensator import make_model as make_mc
#from model.espcn_multiframe2 import make_model as make_espcn
#from approx_huber_loss import Approx_Huber_Loss

def make_model(args):
    return VESPCN(args)

class VESPCN(nn.Module):
    def __init__(self):
        self.name = 'VESPCN'
        self.device = 'cuda'
        super(VESPCN, self).__init__()
        print("Creating VESPCN")
        
        self.mseloss = nn.MSELoss()
        self.huberloss = Approx_Huber_Loss()
        self.motionCompensator = make_mc()
        self.espcn = make_espcn()

    def forward(self, frame_list):
        #frame list中的数据的大小为【N，3，n_colors，H，W】
        # 一个sequence中只有3张图，其中第二张图是目标图。n_sequence=3
        # squeeze frames n_sequence * [N, 1, n_colors, H, W] -> n_sequence * [N, n_colors, H, W]
        #frame_list = [torch.squeeze(frame, dim = 1) for frame in frame_list]
        frame1 = torch.squeeze(frame_list[:,0:1,:,:,:],dim=1)
        frame2 = torch.squeeze(frame_list[:,1:2,:,:,:],dim=1)
        frame3 = torch.squeeze(frame_list[:,2:3,:,:,:],dim=1)
        #这里输入的数据的大小为【N，n_colors，H，W】,输出的数据大小为
        frame1_compensated, flow1 = self.motionCompensator(frame2, frame1)
        frame3_compensated, flow2 = self.motionCompensator(frame2, frame3)
        
        loss_mc_mse = self.mseloss(frame1_compensated, frame2) + self.mseloss(frame3_compensated, frame2)
        loss_mc_huber = self.huberloss(flow1) + self.huberloss(flow2)
        
        #print(frame1_compensated.shape, frame2.shape, frame3_compensated.shape)
        # n_sequence * [N, n_colors, H, W] -> [N, n_sequence * n_colors, H, W]
        lr_frames_cat = torch.cat((frame1_compensated, frame2, frame3_compensated), dim = 1) 
        #print(lr_frames_cat.shape)
        #这里说明了espcn中接受到的数据实际上为[N, n_sequence * n_colors, H, W]
        return self.espcn(lr_frames_cat), loss_mc_mse, loss_mc_huber

class CTTRCN(nn.Module):
    def __init__(self,args):
        self.name = 'CTTRCN'
        self.device = 'cuda'
        if args.cpu:
            self.device = 'cpu' 
        super(CTTRCN, self).__init__()
        print("Creating CTTRCN")   


编写它的训练器

In [10]:
#训练程序需要好好改一下
#下面开始进行训练
#首先需要把X给分成
eps=1e-6
import torch.nn.functional as func

class LossFunc(nn.Module):
    def __init__(self):
        super(LossFunc, self).__init__()

        return

    def forward(self,X,Y):
        #print(X.shape)
        #print(Y.shape)
        df1=torch.add(torch.flatten(torch.flatten(X, start_dim=0,end_dim=1),start_dim=1),-torch.flatten(torch.flatten(Y, start_dim=0,end_dim=1),start_dim=1))
        df1=torch.sqrt(torch.add(torch.sum(df1.mul(df1),dim=1),eps))
        loss=torch.mean(df1)
        return loss
#这里不一样的地方在于:它们的维度是B,C,H,W
#然后得到的任意一张图片，它的处理方法和前面的区别不是特别大,它的顺序是，先再深度层上叠加
def display_demo(dataloader,model,loss_fn,device,t):
    model.eval()
    final_prelist1=[]
    final_prelist2=[]
    for batch, (X, Y) in enumerate(dataloader):
        X=X.to(device)
        Y=Y.to(device)

        pred = model(X)
        #print(pred.shape)
        final_prelist1.append(pred)
        final_prelist2.append(Y)
    output_final(final_prelist1,"new")
    output_final(final_prelist2,"ground truth")
def output_final(final_prelist,name):
    blendB=torch.stack(final_prelist)
    blendB=torch.flatten(blendB, start_dim=0,end_dim=1)
    #print(blendB.shape)
    final_prelist=[]
    height=all_depth
    width=find_height
    true_height_sub=int(height/split_scale_H)
    true_width_sub=int(width/split_scale_W)
    #这里把一个方块内所有深度的都给提取出来。
    #待会还要确保，它生成的过程中，是在深度上提取的。
    for i in range(0,blendB.shape[0],find_width):
        img=blendB[i:i+find_width,:,:,:];
        final_prelist.append(img)
    #这一步得到的是若干张截面图
    final_img=[]
    for i in range(0,find_width):
        img= torch.zeros([1,all_depth,find_height])
        final_img.append(img)
    for i in range(0,find_width):
        for j in range(0,split_scale_H):
            for k in range(0,split_scale_W):
                index=j*split_scale_W+k
                final_img[i][0,j*true_height_sub:(j+1)*true_height_sub,k*true_width_sub:(k+1)*true_width_sub]=final_prelist[index][i,0,:,:]   
    #现在final_img中完全是截面了
    #现在需要把若干张截面转换成slice
    final_slice=[]
    for i in range(all_depth):
        img= torch.zeros([1,find_height,find_width])
        final_slice.append(img)
    for i in range(all_depth):
        for j in range(find_width):    
           # print(final_img[j][0,i,:].shape)
            #print(final_slice[i][0,:,j].shape)
            #print(final_img[j][0,i:i+1,:])
            final_slice[i][0,:,j:j+1]=final_img[j][0,i:i+1,:].reshape((find_height,1))
            
        print(final_slice[i][0,:,:])
    #然后在这里需要进行作图
    filename=name+"display_img_"+str(t)+"_";
    for i in range(5):
        filename_now=filename+str(i)+".jpg"
        output_slice=final_slice[i][0,:,:].cpu().detach().numpy()
        output_slice=output_slice*255
        output_slice_out=np.zeros(output_slice.shape,dtype='uint8')
        for j in range(output_slice.shape[0]):
            for k in range(output_slice.shape[1]):
                output_slice_out[j,k]=int(output_slice[j,k])
        cv2.imwrite(filename_now,output_slice_out)
def output_todrive(filename,index,image):
    filename_now="./out/"+filename+str(index)+".jpg"
    output_slice=image[0,:,:].cpu().detach().numpy()
    output_slice=output_slice*255
    output_slice_out=np.zeros(output_slice.shape,dtype='uint8')
    for j in range(output_slice.shape[0]):
        for k in range(output_slice.shape[1]):
            output_slice_out[j,k]=int(output_slice[j,k])
        cv2.imwrite(filename_now,output_slice_out)
def train(dataloader, model, loss_fn, optimizer,device):
    size = len(dataloader.dataset)
    model.train()
    begin=False
    for batch, (X, Y) in enumerate(dataloader):
        X=X.to(device)
        Y=Y.to(device)
        #print(X.shape,Y.shape)
        pred,_,_ = model(X)
        #print(pred.shape)
        loss = loss_fn(pred,Y[:,1,:,:,:])
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss= loss.item()

        if batch%10==0:
            output_todrive("train_src",batch,X[0,1,:,:,:])
            output_todrive("train_target",batch,Y[0,1,:,:,:])
            output_todrive("train_new",batch,pred[0,:,:,:])
            print(f"loss: {loss:>7f}")
def test(dataloader, model, loss_fn,device,t):
    size = len(dataloader.dataset)
    model.eval()
    begin=False
    final_slice=[]
    for batch, (X, Y) in enumerate(dataloader):
        X=X.to(device)
        Y=Y.to(device)

        pred,_,_ = model(X)
        loss = loss_fn(pred,Y[:,1,:,:,:])
        loss= loss.item()
        if batch%1==0:
            print(f"loss: {loss:>7f}")
        final_slice.append(pred)
    filename="test"+"display_img_"+str(t)+"_";
    for i in range(5):
        filename_now=filename+str(i)+".jpg"
        output_slice=final_slice[0][i,0,:,:].cpu().detach().numpy()
        output_slice=output_slice*255
        output_slice_out=np.zeros(output_slice.shape,dtype='uint8')
        for j in range(output_slice.shape[0]):
            for k in range(output_slice.shape[1]):
                output_slice_out[j,k]=int(output_slice[j,k])
        cv2.imwrite(filename_now,output_slice_out)


# Assuming that we are on a CUDA machine, this should print a CUDA device:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model=VESPCN().to(device)
#定义loss_fn
loss_fn=nn.MSELoss()
#定义optimizer
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
optimizer=torch.optim.Adam(model.parameters(),lr=1e-3, weight_decay=0)
epochs = 100
model.load_state_dict(torch.load('./net/modelVssr1_static'), strict=False);
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    #train(train_loader, model, loss_fn,optimizer,device)
    print("test--------------------------------\n")
    test(test_loader, model, loss_fn,device,t)
    #display_demo(display_loader, model, loss_fn,device,t)
print("Done!")

cpu
Creating VESPCN


RuntimeError: ignored

这里保存这个模型，太凭运气了，一会训练的好一会训练的不好

In [ ]:
# 保存整个网络
torch.save(model, "./net/modelVssr1") 
# 保存网络中的参数, 速度快，占空间少
torch.save(model.state_dict(),"./net/modelVssr1_static")
#--------------------------------------------------
#针对上面一般的保存方法，加载的方法分别是：
#model_dict=torch.load(PATH)
#model_dict=model.load_state_dict(torch.load(PATH))
#https://zhuanlan.zhihu.com/p/38056115
#这里记录一下加载模型的方法，大概就是这样
 #self.motionCompensator.load_state_dict(torch.load('./experiment/model_best_mc.pt'), strict=False)
   #     self.espcn.load_state_dict(torch.load('./experiment/model_best_espcn.pt'), strict=False)

2.查看并修改zero-shot的代码
ZSSR关注的不仅仅是它的模型，还有它的训练方法，不过我们这里不一样的是。
我们的流程是，首先对ZSSR进行预训练，即对每一个Video进行训练，然后在训练过程中，对每一个Video都运用对应的ZSSR模型参数。然后这里的问题在于shuffle的时候怎么知道对应的Video的标号。


下面的这一部分是根据一张图生长一个dataset，需要对其进行修改

In [ ]:
""""
Taken as is from https://github.com/jacobgil/pytorch-zssr/blob/master/source_target_transforms.py
with slight edit in the RandomCrop function to work with tensor instead of pil
"""
import numpy as np
import random
from torchvision.transforms import functional as F
import numbers
import torch

class RandomRotationFromSequence(object):
    """Rotate the image by angle.
    Args:
        degrees (sequence or float or int): Range of degrees to select from.
            If degrees is a number instead of sequence like (min, max), the range of degrees
            will be (-degrees, +degrees).
        resample ({PIL.Image.NEAREST, PIL.Image.BILINEAR, PIL.Image.BICUBIC}, optional):
            An optional resampling filter.
            See http://pillow.readthedocs.io/en/3.4.x/handbook/concepts.html#filters
            If omitted, or if the image has mode "1" or "P", it is set to PIL.Image.NEAREST.
        expand (bool, optional): Optional expansion flag.
            If true, expands the output to make it large enough to hold the entire rotated image.
            If false or omitted, make the output image the same size as the input image.
            Note that the expand flag assumes rotation around the center and no translation.
        center (2-tuple, optional): Optional center of rotation.
            Origin is the upper left corner.
            Default is the center of the image.
    """

    def __init__(self, degrees, resample=False, expand=False, center=None):
        self.degrees = degrees
        self.resample = resample
        self.expand = expand
        self.center = center

    @staticmethod
    def get_params(degrees):
        """Get parameters for ``rotate`` for a random rotation.
        Returns:
            sequence: params to be passed to ``rotate`` for random rotation.
        """
        angle = np.random.choice(degrees)
        return angle

    def __call__(self, data):
        """
            img (PIL Image): Image to be rotated.
        Returns:
            PIL Image: Rotated image.
        """
        hr, lr = data
        angle = self.get_params(self.degrees)
        return F.rotate(hr, angle, self.resample, self.expand, self.center), \
                F.rotate(lr, angle, self.resample, self.expand, self.center)

class RandomHorizontalFlip(object):
    """Horizontally flip the given PIL Image randomly with a probability of 0.5."""

    def __call__(self, data):
        """
        Args:
            img (PIL Image): Image to be flipped.
        Returns:
            PIL Image: Randomly flipped image.
        """
        hr, lr = data
        if random.random() < 0.5:
            return F.hflip(hr), F.hflip(lr)
        return hr, lr

class RandomVerticalFlip(object):
    """Vertically flip the given PIL Image randomly with a probability of 0.5."""

    def __call__(self, data):
        """
        Args:
            img (PIL Image): Image to be flipped.
        Returns:
            PIL Image: Randomly flipped image.
        """
        hr, lr = data
        if random.random() < 0.5:
            return F.vflip(hr), F.vflip(lr)
        return hr, lr

class RandomCrop(object):
    """Crop the given PIL Image at a random location.
    Args:
        size (sequence or int): Desired output size of the crop. If size is an
            int instead of sequence like (h, w), a square crop (size, size) is
            made.
        padding (int or sequence, optional): Optional padding on each border
            of the image. Default is 0, i.e no padding. If a sequence of length
            4 is provided, it is used to pad left, top, right, bottom borders
            respectively.
    """

    def __init__(self, size, padding=0):
        if isinstance(size, numbers.Number):
            self.size = (int(size), int(size))
        else:
            self.size = size
        self.padding = padding

    @staticmethod
    def get_params(data, output_size):
        """Get parameters for ``crop`` for a random crop.
        Args:
            img (PIL Image): Image to be cropped.
            output_size (tuple): Expected output size of the crop.
        Returns:
            tuple: params (i, j, h, w) to be passed to ``crop`` for random crop.
        """
        hr, lr = data
        w, h = hr.shape[-2], hr.shape[-1]
        th, tw = output_size
        if w == tw or h == th:
            return 0, 0, h, w

        if w < tw or h < th:
            th, tw = h//2, w//2

        i = random.randint(0, h - th)
        j = random.randint(0, w - tw)
        return i, j, th, tw

    def __call__(self, data):
        hr, lr = data
        i, j, h, w = self.get_params(data, self.size)
        lr = lr[:, j:j + w, i:i + h]
        hr = hr[:, j:j + w, i:i + h]
        return hr, lr

class ToTensor(object):
    """Convert a ``PIL Image`` or ``numpy.ndarray`` to tensor.
    Converts a PIL Image or numpy.ndarray (H x W x C) in the range
    [0, 255] to a torch.FloatTensor of shape (C x H x W) in the range [0.0, 1.0].
    """

    def __call__(self, data):
        """
        Args:
            pic (PIL Image or numpy.ndarray): Image to be converted to tensor.
        Returns:
            Tensor: Converted image.
        """
        hr, lr = data
        if type(lr) is not torch.Tensor:
            lr = F.to_tensor(lr)

        if type(hr) is not torch.Tensor:
            hr = F.to_tensor(hr)
        return hr, lr

In [ ]:
import glob

from PIL import Image
from torch.utils.data import Sampler, Dataset
from transforms import *
from torchvision.transforms import transforms
from utils import show_images

class ZSSRDataset(Dataset):
    def __init__(self, source_image, sr_factor):
        super(ZSSRDataset, self).__init__()
        self.sr_factor = sr_factor
        self.source_image = source_image
        smaller_side = min(self.source_image.size[0: 2])
        larger_side = max(self.source_image.size[0: 2])

        factors = []
        for i in range(smaller_side // 2, smaller_side + 1):
            downsampled_smaller_side = i
            zoom = float(downsampled_smaller_side) / smaller_side
            if downsampled_smaller_side % self.sr_factor == 0 and not（zoom==0）:
                factors.append(zoom)
        hr_lr_pairs = []
        for zoom in factors:
            hr = self.source_image.resize((int(self.source_image.size[0] * zoom),
                                  int(self.source_image.size[1] * zoom)),
                                 resample=Image.BICUBIC)
            lr = hr.resize((int(hr.size[0] / self.sr_factor),
                            int(hr.size[1])),
                           resample=Image.BICUBIC)

            hr_lr_pairs.append((hr, lr))
        self.hr_lr_pairs = hr_lr_pairs
        self.transform = transforms.Compose([
            RandomRotationFromSequence([0, 90, 180, 270]),
            RandomHorizontalFlip(),
            RandomVerticalFlip()])
    def __getitem__(self, index):
        return self.transform(self.hr_lr_pairs[index])

    def __len__(self):
        return len(self.hr_lr_pairs)

    @classmethod
    def from_image(cls, img, sr_factor):
        pil = transforms.ToPILImage()(transforms.ToTensor()(img))
        return ZSSRDataset(pil, sr_factor)

    def show_pairs(self):
        number_of_pairs = len(self)
        indexes = random.sample(range(number_of_pairs), k=4)
        pairs = [self.__getitem__(index) for index in indexes]
        lrs = [lr for _, lr in pairs]
        hrs = [hr for hr, _ in pairs]
        show_images(lrs + hrs, 2, 4)

    def concat(self, dataset):
        self.hr_lr_pairs += dataset.hr_lr_pairs
        return self

class ZSSRSampler(Sampler):
    def __init__(self, dataset):
        super(ZSSRSampler, self).__init__(dataset)
        self.dataset = dataset
        sizes = np.float32([(hr.size[0] * hr.size[1] / float(
            self.dataset.source_image.size[0] * self.dataset.source_image.size[1])) for hr, lr in self.dataset.hr_lr_pairs])
        self.pair_probabilities = sizes / np.sum(sizes)

    def __iter__(self):
        while True:
            yield random.choices(self.dataset, weights=self.pair_probabilities, k=1)[0]


这里定义的是ZSSR模型，然后和ESPCN模型的思路莫一样，是先插值，然后再使用残差网络进行重建

In [ ]:
class ZSSRModel(nn.Module):
    #在这里的输入应该是要一张图片，然后它的大小和hr图片的大小是一致的，所以在这里没有必要修改它
    def __init__(self,device='cuda'):
        super(ZSSRModel, self).__init__()
        ks=3
        layers_num=8
        Conv2dList=[nn.Conv2d(1, 64, kernel_size=ks, padding=ks // 2, bias=True),]
        layers = [
            Conv2dList[0],
            nn.ReLU()
        ]
        for i in range(layers_num - 2):
            Conv2dList+=[nn.Conv2d(64, 64, kernel_size=ks, padding=ks//2, bias=True),]
            layers += [
                Conv2dList[i+1],
                nn.ReLU()
            ]
        Conv2dList+=[nn.Conv2d(64, 1, kernel_size=ks, padding=ks//2, bias=True),]
        layers += [Conv2dList[layers_num-1],]
        self.model = nn.Sequential(*layers)
        self.device = device
        self.to(device)

    def forward(self, X):
      #这里使用的是残差网络
        return self.model(X) + X
    def freezing(self)：
        for o in Conv2dList:
            for para in o.parameters():
                para.requires_grad = False


这一个是对ZSSR模型的训练函数
在这里我仍然按照它的想法进行，因为它的对upsampling过后的结果使用残差网络是有一定的道理的。因为在训练数据不大的时候，尽可能在插值的基础上进行改造可能要合理的多。
所以后面我还需要做一步从hr到feature map的过程


其中：data_sampler就是通过运行上面的步骤得到的采样器

在训练的过程中，它的上采样函数仍然是一个神经网络。，就是ESPCN网络，我

In [ ]:
def bicubic_upsample(lr, scale):
    lr = lr.resize([lr.size[0] * scale, lr.size[1]],
                   resample=Image.BICUBIC)
    return transforms.ToTensor()(lr)
def train_single_img(lr_img,
                     model,
                    optimizer,
                    num_batches,
                    device="cuda"):
#在调用的时候，这里的num_batches为
    dataset = ZSSRDataset(lr_img, scale)
    data_sampler = ZSSRSampler(dataset)
    model.train()
    avg_loss = 0
    l1_loss = F.l1_loss
    trans = transforms.Compose([
        ToTensor(),
        RandomCrop(config["crop_size"])
    ])
    for iter, (hr, lr) in enumerate(data_sampler):
        optimizer.zero_grad()
        scale = hr.size[0] // lr.size[0]
        lr = bicubic_upsample(lr, scale)
        hr, lr = trans((hr, lr))#没有必要裁剪。因为它给的那个裁剪的范围有点大，我们的达不到那个范围
        #看样子这里一个batch的大小应该是1
        hr, lr = hr.unsqueeze(0).to(device), lr.unsqueeze(0).to(device)
        hr_pred = model(lr)
        loss = l1_loss(hr_pred, hr)

        avg_loss += loss.item()
        loss.backward()
        optimizer.step()

        if iter > num_batches:
            print('Done training.')
            avg_loss /= iter
            print(f'Avg training loss is {avg_loss}')
            break

然后开始写融合的以及还原feature map的部分

这里是需要仿照前面的函数写一个逆函数

In [ ]:
class myInvPixelShuffle(nn.Module):
    def __init__(self,scale):
        super(myPixelShuffle, self).__init__()
        self.scale=scale
    def forward(self,sr_img):
      #sr_img的形状是【B，1，r*H，W】，我需要先将其还原为【B，r，H，W】
        final_list=[]
        for i in range(self.scale):
            final_list.append([])
        for i in range(fmap.shape[2]):
            index=i%self.scale
            final_list[index].append(sr_img[:,:,i:i+1,:])
        for i in range(self.scale):
            torch.cat(final_list[index],dim=2)
        return   torch.cat(final_list,dim=1)
def myInvpixel_shuffle(fmap,scale):
	#这里fmap的维度是:[B,sizeO=scale,H,W],我需要做的是把他们H给连接起来
    final_list=[]
    for i in range(scale):
        final_list.append([])
    for i in range(fmap.shape[2]):
        index=i%scale
        final_list[index].append(sr_img[:,:,i:i+1,:])
    for i in range(scale):
        torch.cat(final_list[index],dim=2)
    return   torch.cat(final_list,dim=1)

这一个类就是融合他们的类

In [ ]:
class BlendModel(nn.Module):
    #在这里的输入应该是要一张图片，然后它的大小和hr图片的大小是一致的，所以在这里没有必要修改它
    def __init__(self,device='cuda'):
        super(BlendModel, self).__init__()
        ks=3
        layers_num=8
        layers = [
            nn.Conv2d(4*scale, 64, kernel_size=ks, padding=ks // 2, bias=True),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=ks, padding=ks // 2, bias=True),
            nn.ReLU(),
            nn.Conv2d(64, scale, kernel_size=ks, padding=ks // 2, bias=True),
            nn.ReLU(),
            myPixelShuffle(scale)
        ]
        self.model = nn.Sequential(*layers)
        self.device = device
        self.to(device)

    def forward(self, X):
      #这里使用的是残差网络
        return self.model(X)

这里亟需解决的问题是，在生成dataset过后，因为会shuffle我应该怎么去获取他们的序号